# 1. Data Gathering 

In [1]:
import numpy as np
import pandas as pd
import ast # for converting string to list 

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
# credits.head(1)['crew'].values

## Merging both dataframes

we can merge both dataset on the basis of either title or id

In [6]:
movies.shape

(4803, 20)

In [7]:
credits.shape
 

(4803, 4)

In [ ]:
movies = movies.merge(credits,on='title')
movies.shape

# Preprocessing 

we will be creating a content based recommender system so we will be using only that columns which is helping us in creaditing tags so we are going to ask question before selecting a columns that :
    **Where this column is helpfull in creating tags ?** :
        

In [ ]:
movies.head(1).columns

### columns we will be kepping 
* genres
* id
* keywords
* overview 
* title
* cast 
* crew
* release date





In [ ]:
movies = movies[['movie_id','title','genres','keywords', 'overview','cast','crew']]

In [ ]:
# movies.info()

**we will be creating new data frame from this dataset having three columns :**
* movie id
* title
* tag 

now we need to creat this tag columns

In [ ]:
# Missing data

In [ ]:
# movies[movies.isnull()==True]
movies.isnull().sum()

there is 3 movies in which overoview is missing, since 3 is not big number we will drop this

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0]['genres']

it is list of dictionary we need to bring all keywords into a list 
for example :
    ['action','adventure','fantasy','scify']

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
        

In [ ]:
movies['genres']=movies.genres.apply(convert)

In [ ]:
movies['keywords']=movies.keywords.apply(convert)

In [ ]:
movies.head(1)

In [ ]:
def convert_cast(obj):
    count=0
    l=[]
    for j in ast.literal_eval(obj):
        if count!=3:
            l.append(j['name'])
            count+=1
        else:
            break
    return l

In [ ]:
movies['cast']=movies['cast'].apply(convert_cast)

In [ ]:
 movies.head(1)

In [ ]:
def convert_crew(obj):
    director=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            director.append(i['name'])
    return director

In [ ]:
movies['crew']=movies['crew'].apply(convert_crew)

In [ ]:
movies.head(1)

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split()) #for i in movies['overview']: print(i.split(" ")) same code

now concatenating genres , keywords , overview and crew , cast columns into one single list then converting that list into one string

In [ ]:
def transform(obj):
    l = []
    for i in obj:
        l.append("".join(i.split(" ")))
    return l


# you can use this alsp movies.apply() lambda x:[i.replace(" ".'') for i in x]

In [ ]:
movies['cast']=movies['cast'].apply(transform)
movies['crew']=movies['crew'].apply(transform)
movies['genres']=movies['genres'].apply(transform)
movies['keywords']=movies['keywords'].apply(transform)
movies['overview']=movies['overview'].apply(transform)

In [ ]:
movies.head(5)

In [ ]:
movies['tags'] = movies.keywords + movies.genres + movies.overview + movies.cast + movies.crew

In [ ]:
new_df = movies[['movie_id','title','tags']]


In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

# Text Vectorization 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,stop_words='english')


In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray() # this will give sparse matrix convert it into array

In [ ]:
vectors.shape

In [ ]:
vectors[0]

we can see the most frequents words using one feature get_feature name

In [ ]:
cv.get_feature_names_out()

# Stemming :
technique to make similar function , same

In [ ]:
pip install nltk

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [ ]:
def stem(obj):
    y = []
    for i in obj.split():
        y.append(ps.stem(i))
    return " ".join(y)
        

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

* The CountVectorizer is a text vectorization technique that converts a collection of text documents into a matrix of token counts (i.e., the frequency of each word in the document).

* max_features=5000: This parameter limits the vocabulary size to the top 5000 most frequent words. It ensures that the resulting vector representation is not too sparse and reduces computational complexity.

* stop_words='english': This parameter specifies that common English words (e.g., "the," "is," "and") should be excluded from the vocabulary since they often carry little

* We use the fit_transform method of the CountVectorizer instance (cv) to perform the vectorization.

* The fit_transform method takes the text data from the 'tags' column of the DataFrame new_df and converts it into a document-term matrix.

* The resulting vector is a 2D array representing the numerical vector representation of the text data, where each row corresponds to a document (movie) and each column corresponds to a word in the vocabula

In [ ]:
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()


In [ ]:
cv.get_feature_names_out()

# Similarity 

we will not be calculating euclidean distance **curse of Dimensionality** because in higher dimension ecludiean distance performance decreases thats why we will be using cosine distance in which we are calculating angle between the two vector and those vector whose angle will be less will be the closest

cos(0) == 1 if both vector are same then angle will be zero value will be one
cos(90) == 0 if they are perpendicular then distance will be zero 

**Distance is inversly propotional to similarity**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
sorted(similarity[0],reverse=True)[1:5]

In [ ]:
similarity[0]

In [ ]:
list(enumerate(similarity[0]))

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True)

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key= lambda x:x[1])[1:6]
movie='Avatar'
new_df[new_df['title']==movie].index[0]

 

the code snippet is used to sort the similarity scores for the first movie (the row at index 0 of the similarity matrix) in descending order and select the top 5 similar movies (excluding the first movie itself) based on their similarity scores. The result is a list of tuples, where each tuple contains the index of the movie and its corresponding similarity score.

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(similarity[0])),reverse=True,key= lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]]['title'])

In [ ]:
recommend("Titan A.E")

In [ ]:
new_df['title'].values

In [ ]:
new_df

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb')) #write binary mode

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
x={"adult":False,"backdrop_path":"/hZkgoQYus5vegHoetLkCJzb17zJ.jpg","belongs_to_collection":None,"budget":63000000,"genres":[{"id":18,"name":"Drama"},{"id":53,"name":"Thriller"},{"id":35,"name":"Comedy"}],"homepage":"http://www.foxmovies.com/movies/fight-club","id":550,"imdb_id":"tt0137523","original_language":"en","original_title":"Fight Club","overview":"A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground \"fight clubs\" forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.","popularity":97.396,"poster_path":"/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg","production_companies":[{"id":508,"logo_path":"/7cxRWzi4LsVm4Utfpr1hfARNurT.png","name":"Regency Enterprises","origin_country":"US"},{"id":711,"logo_path":"/tEiIH5QesdheJmDAqQwvtN60727.png","name":"Fox 2000 Pictures","origin_country":"US"},{"id":4700,"logo_path":"/A32wmjrs9Psf4zw0uaixF0GXfxq.png","name":"The Linson Company","origin_country":"US"},{"id":20555,"logo_path":"/hD8yEGUBlHOcfHYbujp71vD8gZp.png","name":"Taurus Film","origin_country":"DE"},{"id":54051,"logo_path":None,"name":"Atman Entertainment","origin_country":""},{"id":54052,"logo_path":None,"name":"Knickerbocker Films","origin_country":"US"}],"production_countries":[{"iso_3166_1":"DE","name":"Germany"},{"iso_3166_1":"US","name":"United States of America"}],"release_date":"1999-10-15","revenue":100853753,"runtime":139,"spoken_languages":[{"english_name":"English","iso_639_1":"en","name":"English"}],"status":"Released","tagline":"Mischief. Mayhem. Soap.","title":"Fight Club","video":False,"vote_average":8.428,"vote_count":25236}

In [ ]:
x['poster_path']

In [ ]:
https://api.themoviedb.org/3/movie/{}?api_key=2234cda012e52526ee792fda591639f4&language=en-US

In [ ]:
2234cda012e52526ee792fda591639f4

In [ ]:

https://api.themoviedb.org/3/movie/550?api_key=2234cda012e52526ee792fda591639f4&language=en-US